In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("rafalposwiata/deproberta-large-depression")
model = AutoModelForSequenceClassification.from_pretrained("rafalposwiata/deproberta-large-depression")

In [ ]:
import torch
import numpy as np


from transformers import pipeline
question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

context = """ okay arms chat shell let know guys done okay alright yeah im okay im chicago illinois husbands work 84 easy took three years im move moved away family uncomfortable didnt know anyone even though make friends felt place often would like used go back least year last years thats diminish would say like every 3 years weather love weather beach know ive long people nice people people people yeah also would say economy difficult make los angeles business computer graphics im working right im working retail couldnt find work settle right tough would like human resource representative like talk people much understand right know lot subject go im interested finding people im sitting next somebody else strike conversation find know id like talk people enjoy im quiet rude people people harm children elderly makes angry war control temper dont problem get quiet feel angry try think act last time argued someone month ago daughter arguing therapy cancer dumb wasnt telling enough wasnt giving enough information made angry frustrated angry yeah hard watching children fail theyre trying hard seeing hurt trying prepare wont listen advice feel like sometimes need trust love children stop playing granddaughter month ago excited happy cares gone wasnt thinking anything except fun good time yeah best yes 5 years ago couldnt function couldnt drive couldnt sleep couldnt eat couldnt anything completely shut felt like felt like looking window life going outside window stuck behind glass iii couldnt participate knew something wasnt right lately ive feeling depressed havent good night sleep year would say regular pattern maybe sleep hour 3 hour intervals keep waking never quite get deep sleep groggy energy hard difficult work difficult concentrate make mistakes forget lot yes yes im therapy right could afford continue im sorry im sure yes feel relaxed feel like worries lifted think clearer using tools ive learned therapy children dont know friendly think horrible feel children neglected today fault parents think dumb think society forgot takes village raise child nobody wants pitch one wants help anymore ones helping neighbor sad dont know im eating dinner family regret spending enough time education children young needed yeah needed take care im sure could seek help grandparents im introduced church know letting play home play garden showed lot love felt like cared listening music watching funny program tv going school working go dinner friend year ago friend work close youre welcome bye okay """

questions = ['how easy is it for you to get a good night sleep?', 'have you noticed any changes in your behavior or thoughts lately?',
             'have you been diagnosed with depression?', 'are you intimate with your family?', 'how have you been feeling lately?',
             'do you consider yourself an introvert', 'have you ever been diagnosed with ptsd?', 'do you feel like therapy is useful?']

# create a feature vector for the context
feature_vector = []

for question in questions:
    result = question_answerer(question=question, context=context)
    print(f"Answer: '{question}':'{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")
    # classify the answer
    deproberta_input_ids = torch.tensor(tokenizer.encode(result['answer'], add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    deproberta_outputs = model(deproberta_input_ids)
    deproberta_logits = deproberta_outputs[0]
    deproberta_probs = torch.nn.functional.softmax(deproberta_logits, dim=1)
    print(f"Depression probability: {deproberta_probs[0][1]}")
    if result['score'] > 0.1:
        feature_vector.append(deproberta_probs[0][1])
    else:
        # append a tensor with a value of 0
        feature_vector.append(torch.tensor(0))
        

# convert the feature vector items from tensor to a value
feature_vector = [x.item() for x in feature_vector]

# print the feature vector
print(feature_vector)

